In [ ]:
!pip install -q transformers datasets
!pip install -q peft
!pip install -q -i https://test.pypi.org/simple/ bitsandbytes
!pip install -q trl
!pip install -q accelerate

# Import

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    GenerationConfig
)
import transformers
from datasets import load_dataset
from datasets import Dataset


from trl import SFTTrainer

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from peft import PeftModel

import torch
import json, os

import warnings
warnings.filterwarnings('ignore')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# Model Load

In [ ]:
model_name = 'beomi/Yi-Ko-6B'

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)
model.config.pretraining_tp = 1
model.config.use_cache = False

# LoRA Model

In [ ]:
lora_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="CAUSAL_LM"
)

In [ ]:
model_add_lora = get_peft_model(model, lora_config)

In [ ]:
model_add_lora.print_trainable_parameters()

trainable params: 3,276,800 || all params: 6,182,801,408 || trainable%: 0.05299862932294914


# Dataset

In [ ]:
ds = Dataset.from_csv("/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/multi_turn_dataset.csv")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['질문자', '전문가'],
    num_rows: 149578
})

In [ ]:
ds = ds.map(
    lambda x: {'text' : f"""User : {x['질문자']} \n\n Assistant : {x['전문가']}"""}
)

Map:   0%|          | 0/149578 [00:00<?, ? examples/s]

# Train

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
training_args=TrainingArguments(
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    max_steps = 20000,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=10,
    save_total_limit=2,
    output_dir="/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/Yi_ko_6b_v2/",
)

In [ ]:
trainer = SFTTrainer(
    model=model_add_lora,
    train_dataset=ds,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args,
    packing= False,
)

Map:   0%|          | 0/149578 [00:00<?, ? examples/s]

In [ ]:
trainer.train("/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/Yi_ko_6b/checkpoint-2000")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2010,2.191900
2020,2.237800
2030,2.247000
2040,2.219900
2050,2.244300
2060,2.475500
2070,2.427600
2080,2.307500
2090,2.289600
2100,2.355800


FailedPreconditionError: ignored